## Web Scraping(From Amazon)
## My scraper extract the data from Amazon
## The user have to just enter the item and number of pages from which he/she want to extract data.


## Importing libraries

In [1]:
from bs4 import BeautifulSoup

from selenium import webdriver

import pandas as pd

In [2]:
driver = webdriver.Chrome(r'C:\Users\Uzair Ahmad\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe')

## Getting Urls

In [3]:
def get_url(item_search,pages):
    url_list = []
    url = 'https://www.amazon.com/s?k={}&page='
    item_search = item_search.strip()
    item_search = item_search.replace(' ','+')
    for page_num in range(pages):
        url_list.append(url.format(item_search)+str(page_num+1))

    return url_list

## This is for user
## Enter name of item and number of pages

In [4]:
urls=get_url('laptop',1) #Enter item and number of pages

In [5]:
urls

['https://www.amazon.com/s?k=laptop&page=1']

## Scraper

In [15]:
def scraper():
    data=[]
    for url in urls:  # urls of different pages
        driver.get(url) # driver of first page
        soup=BeautifulSoup(driver.page_source,'html.parser') # soup for different pages
        results = soup.find_all("div",{"data-component-type":"s-search-result"}) # content of different pages
        for item in range(len(results)): # looping through a page to get all items
            dic={}
            
            
            item_url='https://www.amazon.com'+results[item].a.get('href') # url of an item page
            driver.get(item_url) #driver for an item contents
            soup1=BeautifulSoup(driver.page_source,'html.parser') # soup for an item page
            item_specs_heading = soup1.find_all('th',{'class':'a-color-secondary a-size-base prodDetSectionEntry'})
            item_specs = soup1.find_all('td',{'class':'a-size-base prodDetAttrValue'}) # getting content of item page

            for num in range(len(item_specs)):
                try:
                    dic[item_specs_heading[num].text.strip()] = item_specs[num].text.strip().replace('\u200e','')
                except:
                    dic[item_specs_heading[num].text.strip()] = None

                
            try:
                dic['Price'] = soup1.find('span',{'id':'priceblock_ourprice'}).text
            except:
                dic['Price']=None
            
            try:
                dic["Ratings"] = soup1.find('span',{'id':'acrCustomerReviewText'}).text
            except:
                dic["Ratings"] = None
            
            try:
                dic["Reviews"] = soup1.find('span',{'class':'a-icon-alt'}).text
            except:
                dic["Reviews"] = None
                              
            try:
                dic["Reviews Link"] = 'https://www.amazon.com'+soup1.find('a',{'class':'a-link-normal'}).get('href')
            except:
                dic["Reviews Link"] = None
                
            dic = list(dic.items())
            dic.sort()
            dic = dict(dic)
            data.append(dic)
    driver.close()
    return data

## Making data frame from data extracted data

In [16]:
extracted_data=scraper()

In [17]:
extracted_data

[{'ASIN': 'B08YDLCXR4',
  'Average Battery Life (in hours)': '19.5 Hours',
  'Batteries': '2 Lithium ion batteries required. (included)',
  'Brand': 'LG',
  'Card Description': 'Integrated',
  'Chipset Brand': 'Intel',
  'Color': 'Black',
  'Computer Memory Type': 'DDR4 SDRAM',
  'Customer Reviews': 'March 9, 2021',
  'Flash Memory Size': '16',
  'Hard Drive': '2 TB SSD',
  'Hard Drive Interface': 'Solid State',
  'Hardware Platform': 'PC',
  'Item Dimensions  LxWxH': '14.97 x 10.24 x 0.7 inches',
  'Item Weight': '2.98 pounds',
  'Item model number': '17Z90P-K.AAB9U1',
  'Max Screen Resolution': '2560 x 1600 Pixels',
  'Number of USB 3.0 Ports': '1',
  'Operating System': 'Windows 10 Home',
  'Optical Drive Type': 'No Optical Drive',
  'Power Source': 'Battery Powered',
  'Price': '$1,796.99',
  'Processor': '4.7 GHz core_i7',
  'Processor Brand': 'Intel',
  'Processor Count': '4',
  'Product Dimensions': '14.97 x 10.24 x 0.7 inches',
  'RAM': '16 GB LPDDR4',
  'Ratings': '44 ratings'

In [18]:
df=pd.DataFrame(extracted_data)
df

,ASIN,Average Battery Life (in hours),Batteries,Brand,Card Description,Chipset Brand,Color,Computer Memory Type,Customer Reviews,Flash Memory Size,...,Series,Standing screen display size,Wireless Type,Graphics Coprocessor,Memory Speed,Number of USB 2.0 Ports,Voltage,Date First Available,Manufacturer,Package Dimensions
0,B08YDLCXR4,19.5 Hours,2 Lithium ion batteries required. (included),LG,Integrated,Intel,Black,DDR4 SDRAM,"March 9, 2021",16,...,LG gram,17 Inches,802.11ax,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,B08SW1G8FW,25.5 Hours,2 Lithium ion batteries required. (included),LG,Integrated,Intel,Black,DDR4 SDRAM,"January 13, 2021",16,...,14Z90P-K.AAB8U1,14 Inches,802.11ax,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,B08YKGZF7P,12 Hours,1 Lithium ion batteries required. (included),HP,Integrated,Intel,Ceramic White,NaN,"March 28, 2021",4,...,14a-na0021nr,14 Inches,"802.11a/b/g/n/ac, Bluetooth",Intel UHD Graphics 600,2400 MHz,NaN,NaN,NaN,NaN,NaN
3,B086KKKT15,5 Hours,1 Lithium ion batteries required. (included),Acer,Integrated,AMD,Silver,DDR4 SDRAM,"April 27, 2020",512,...,SF314-42-R9YN,14 Inches,802.11ax,NaN,4.1 GHz,1,NaN,NaN,NaN,NaN
4,B08HN1WDWZ,8 Hours,1 Lithium Polymer batteries required. (included),Acer,Integrated,Intel,Silver,DDR4 SDRAM,"November 15, 2020",128,...,A515-55-35SE,15.6 Inches,Bluetooth,NaN,3.4 GHz,1,NaN,NaN,NaN,NaN
5,B08VKNVDDR,10 Hours,1 Lithium Polymer batteries required. (included),Acer,Integrated,AMD,Silver,DDR4 SDRAM,"May 12, 2021",128,...,A515-46-R14K,15.6 Inches,Bluetooth,NaN,3.5 GHz,1,NaN,NaN,NaN,NaN
6,B096YGTBG2,NaN,NaN,HP,NaN,NaN,Silver,DDR SDRAM,"June 9, 2021",NaN,...,Laptop 17,17 Inches,Bluetooth,Intel Iris Xe Graphics,NaN,NaN,NaN,NaN,NaN,NaN
7,B08XN47X4V,NaN,1 Lithium ion batteries required. (included),Dell,NaN,NaN,White,DDR4 SDRAM,"February 26, 2021",NaN,...,Dell Inspiron,15.6 Inches,Bluetooth,AMD Radeon Graphics,3.5 GHz,NaN,NaN,NaN,NaN,NaN
8,B096CVCDP2,NaN,NaN,HP,NaN,NaN,Rose Gold,DDR4 SDRAM,"June 1, 2021",64 GB,...,Premium,14 Inches,"Bluetooth, 802.11a",Intel UHD Graphics,2.8 GHz,NaN,NaN,NaN,NaN,NaN
9,B081V6W99V,12 Hours,1 Lithium Metal batteries required. (included),ASUS,Integrated,Intel,Black,DDR4 SDRAM,"October 5, 2020",64,...,L210MA-DB01,11.6 Inches,802.11ac,NaN,2.8 GHz,1,19 Volts,NaN,NaN,NaN


In [20]:
df.to_csv('AmazonLaptopExtractedData.csv')